In [10]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.table import Table
import matplotlib.pyplot as plt
import warnings

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [11]:
# User input: run folder name
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

lightcurve_files = sorted(run_folder.glob("*_lightcurve.csv"))
print(f"Found {len(lightcurve_files)} lightcurve CSV(s) in {run_folder}")
for f in lightcurve_files:
    print(f"{f.name}")

Enter the run folder name: test3
Found 46 lightcurve CSV(s) in /Users/david/Code/msc/runs/test3
ZTF19aailqir_lightcurve.csv
ZTF19aailqli_lightcurve.csv
ZTF19aailteb_lightcurve.csv
ZTF19aailuvj_lightcurve.csv
ZTF19aajwdqt_lightcurve.csv
ZTF19aajwhbc_lightcurve.csv
ZTF19aajxhwa_lightcurve.csv
ZTF19aajxwnz_lightcurve.csv
ZTF19aakiwxx_lightcurve.csv
ZTF19aakiyly_lightcurve.csv
ZTF19aakjbej_lightcurve.csv
ZTF19aaklbok_lightcurve.csv
ZTF19aakljys_lightcurve.csv
ZTF19aaklqod_lightcurve.csv
ZTF19aaklquv_lightcurve.csv
ZTF19aaknign_lightcurve.csv
ZTF19aakocxy_lightcurve.csv
ZTF19aakpfax_lightcurve.csv
ZTF19aaksrgj_lightcurve.csv
ZTF19aakthjl_lightcurve.csv
ZTF19aakwifv_lightcurve.csv
ZTF19aakzwao_lightcurve.csv
ZTF19aalaxsa_lightcurve.csv
ZTF19aalcwjt_lightcurve.csv
ZTF19aalcyfd_lightcurve.csv
ZTF19aaldqvh_lightcurve.csv
ZTF19aaleptm_lightcurve.csv
ZTF19aaljcsu_lightcurve.csv
ZTF19aalpoff_lightcurve.csv
ZTF19aaluprf_lightcurve.csv
ZTF19aalveag_lightcurve.csv
ZTF19aalxcyo_lightcurve.csv
ZTF19aal

In [12]:
filter_map = {"g": "ztfg", "r": "ztfr"}
# Redshift per object from ztf_cleansed.csv; fit t0, x0, x1, c only
fit_params = ["t0", "x0", "x1", "c"]
bounds = {"x1": (-3, 3), "c": (-0.3, 0.3)}

all_results = []

In [13]:
# Load ztf_cleansed.csv
ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_meta = pd.read_csv(ztf_cleansed_path)
ztf_meta["ZTFID"] = ztf_meta["ZTFID"].astype(str).str.strip()

total_files = len(lightcurve_files)
completed = 0

for idx, lc_path in enumerate(lightcurve_files, 1):
    obj_id = lc_path.stem.replace("_lightcurve", "")
    print(f"[{idx}/{total_files}] Processing {obj_id}....")

    # Load cleaned lightcurve from downloadLasair (MJD, filter, forced_ujy, forced_ujy_error)
    df = pd.read_csv(lc_path)
    df["MJD"] = pd.to_numeric(df["MJD"], errors="coerce")
    df = df.dropna(subset=["MJD", "filter", "forced_ujy", "forced_ujy_error"])
    df["filter"] = df["filter"].astype(str).str.strip().str.lower()
    df = df[df["forced_ujy"].gt(0) & df["forced_ujy_error"].gt(0)].copy()

    if len(df) < 5:
        print(f"  Skip {obj_id}: too few points ({len(df)})")
        continue

    bands = [filter_map.get(f, f"ztf{f}") for f in df["filter"].values]
    data = Table({
        "time": df["MJD"].values,
        "band": bands,
        "flux": df["forced_ujy"].values,
        "fluxerr": df["forced_ujy_error"].values,
        "zp": np.full(len(df), 23.9),
        "zpsys": np.array(["ab"] * len(df)),
    })

    # Look up obj_id in the ZTFID column
    redshift = float(ztf_meta.loc[ztf_meta["ZTFID"] == obj_id, "redshift"].values[0])

    model = sncosmo.Model(source="salt2")
    model.set(z=redshift)

    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            result, fitted_model = sncosmo.fit_lc(data, model, fit_params, bounds=bounds)
    except Exception as e:
        print(f"  Fit failed: {e}")
        continue

    row = {"object_id": obj_id, "chisq": result.chisq, "ndof": result.ndof, "z": redshift}
    for name, val in zip(result.param_names, result.parameters):
        row[name] = val
    if result.errors is not None:
        for name in result.param_names:
            if name in result.errors:
                row[f"{name}_err"] = result.errors[name]
    all_results.append(row)

    fig = sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
    plt.gcf().suptitle(obj_id)
    out_plot = run_folder / f"{lc_path.stem}_salt2.png"
    plt.savefig(out_plot, dpi=150, bbox_inches="tight")
    plt.close()
    completed += 1
    print(f"  Saved {out_plot.name}  ({completed}/{total_files} completed)")

[1/46] Processing ZTF19aailqir....
  Saved ZTF19aailqir_lightcurve_salt2.png  (1/46 completed)
[2/46] Processing ZTF19aailqli....
  Saved ZTF19aailqli_lightcurve_salt2.png  (2/46 completed)
[3/46] Processing ZTF19aailteb....
  Saved ZTF19aailteb_lightcurve_salt2.png  (3/46 completed)
[4/46] Processing ZTF19aailuvj....
  Saved ZTF19aailuvj_lightcurve_salt2.png  (4/46 completed)
[5/46] Processing ZTF19aajwdqt....
  Saved ZTF19aajwdqt_lightcurve_salt2.png  (5/46 completed)
[6/46] Processing ZTF19aajwhbc....
  Saved ZTF19aajwhbc_lightcurve_salt2.png  (6/46 completed)
[7/46] Processing ZTF19aajxhwa....
  Saved ZTF19aajxhwa_lightcurve_salt2.png  (7/46 completed)
[8/46] Processing ZTF19aajxwnz....
  Saved ZTF19aajxwnz_lightcurve_salt2.png  (8/46 completed)
[9/46] Processing ZTF19aakiwxx....
  Saved ZTF19aakiwxx_lightcurve_salt2.png  (9/46 completed)
[10/46] Processing ZTF19aakiyly....
  Saved ZTF19aakiyly_lightcurve_salt2.png  (10/46 completed)
[11/46] Processing ZTF19aakjbej....
  Saved ZTF1

In [14]:
# Output CSV of sncosmo parameters
if all_results:
    params_df = pd.DataFrame(all_results)
    out_csv = run_folder / "sncosmo_parameters.csv"
    params_df.to_csv(out_csv, index=False)
    print(f"Saved sncosmo parameters to {out_csv}")
    display(params_df)
else:
    print("No successful fits to write.")

Saved sncosmo parameters to /Users/david/Code/msc/runs/test3/sncosmo_parameters.csv


,object_id,chisq,ndof,z,t0,x0,x1,c,t0_err,x0_err,x1_err,c_err
0,ZTF19aailqir,1.939871,7,0.05434,58529.240320,0.000424,0.005314,0.300000,0.526847,0.000020,0.719291,0.061484
1,ZTF19aailqli,86.069495,19,0.05800,58535.586782,0.001352,0.198582,-0.025923,0.240896,0.000089,0.368625,0.047729
2,ZTF19aailteb,16.734382,8,0.07800,58524.495904,0.000725,2.999996,-0.022828,0.836042,0.000038,0.979059,0.053891
3,ZTF19aailuvj,4.449338,14,0.07100,58536.548154,0.000838,0.470822,-0.073357,0.313527,0.000055,0.422183,0.047706
4,ZTF19aajwdqt,16.859893,13,0.08800,58538.834933,0.000714,3.000000,0.074628,0.333026,0.000034,0.171552,0.046395
5,ZTF19aajwhbc,0.253652,1,0.10000,58536.914177,0.000537,-0.573256,-0.126057,2.095029,0.000330,4.026896,0.355833
6,ZTF19aajxhwa,14.490219,18,0.03300,58539.507062,0.001702,0.409510,0.144662,0.229428,0.000050,0.299397,0.028819
7,ZTF19aajxwnz,2948.382560,73,0.01500,58544.707934,0.011558,0.855165,0.199285,0.066067,0.000184,0.084173,0.011170
8,ZTF19aakiwxx,6.580320,30,0.04500,58537.073872,0.001960,-1.003797,-0.047183,0.172470,0.000068,0.175515,0.028093
9,ZTF19aakiyly,57.219497,17,0.07000,58539.562703,0.000959,0.667547,-0.045161,0.307987,0.000044,0.304788,0.038792
